In [1]:
# 라이브러리

import os
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix

import warnings
warnings.filterwarnings("ignore")

In [2]:
path = './movielens_data/'
ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), index_col='movieId', encoding='utf-8')
tags_df = pd.read_csv(os.path.join(path, 'tags.csv'), encoding='utf-8')

In [3]:
print(ratings_df.shape)
print(ratings_df.head())

(100836, 4)
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [4]:
num_users = ratings_df['userId'].unique()
num_movies = ratings_df['movieId'].unique()

print("총 유저 수: ", len(num_users))
print("총 영화 수: ", len(num_movies))

총 유저 수:  610
총 영화 수:  9724


---

In [5]:
# pivot ratings into movie features
user_movie_matrix = ratings_df.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)
 
# convert dataframe of movie features to scipy sparse matrix
sparse_mat = csr_matrix(user_movie_matrix.values)

In [6]:
print(sparse_mat)

  (0, 0)	4.0
  (0, 4)	4.0
  (0, 6)	4.5
  (0, 14)	2.5
  (0, 16)	4.5
  (0, 17)	3.5
  (0, 18)	4.0
  (0, 20)	3.5
  (0, 26)	3.0
  (0, 30)	5.0
  (0, 31)	3.0
  (0, 32)	3.0
  (0, 39)	5.0
  (0, 42)	5.0
  (0, 43)	3.0
  (0, 44)	4.0
  (0, 45)	5.0
  (0, 49)	3.0
  (0, 53)	3.0
  (0, 56)	5.0
  (0, 62)	5.0
  (0, 63)	4.0
  (0, 65)	4.0
  (0, 67)	2.5
  (0, 70)	5.0
  :	:
  (9700, 337)	2.5
  (9701, 337)	3.0
  (9702, 183)	4.0
  (9702, 247)	3.5
  (9703, 317)	2.5
  (9704, 209)	1.0
  (9705, 461)	2.5
  (9706, 49)	3.5
  (9707, 337)	1.5
  (9708, 337)	4.0
  (9709, 337)	1.0
  (9710, 337)	1.5
  (9711, 337)	1.0
  (9712, 337)	1.0
  (9713, 183)	4.5
  (9714, 183)	3.5
  (9715, 183)	3.0
  (9716, 183)	4.0
  (9717, 183)	4.0
  (9718, 183)	3.5
  (9719, 183)	4.0
  (9720, 183)	3.5
  (9721, 183)	3.5
  (9722, 183)	3.5
  (9723, 330)	4.0


In [7]:
user_info_df = pd.DataFrame(data = [sum(list(user_movie_matrix[int(x)].value_counts())[1:]) for x in user_movie_matrix.columns],
                            index = user_movie_matrix.columns, 
                            columns=['movies_rated'])

In [8]:
user_info_df.head()

,movies_rated
userId,
1,232
2,29
3,39
4,216
5,44


In [9]:
movie_info_df = pd.DataFrame(data = [sum(list(user_movie_matrix.loc[int(x)].value_counts())[1:]) for x in user_movie_matrix.index],
                             index = user_movie_matrix.index, columns=['users_rated'])

In [10]:
movie_info_df.head()

,users_rated
movieId,
1,215
2,110
3,52
4,7
5,49


---

In [11]:
train_df, test_df = train_test_split(ratings_df, test_size=0.2, random_state=1234)

In [12]:
print(train_df.shape)
print(test_df.shape)

(80668, 4)
(20168, 4)


In [13]:
# test set에는 존재하지만, train set에는 없는 영화 또는 사용자 비율

# userId
print("사용자: ",len(list(set(test_df['userId'].unique()) - set(train_df['userId'].unique()))))

# movieId
print("영화: ", len(list(set(test_df['movieId'].unique()) - set(train_df['movieId'].unique()))))
print("test set의 전체 영화 수: ",  len(test_df['movieId'].unique()))

사용자:  0
영화:  786
test set의 전체 영화 수:  5171


In [14]:
movies_not_included = list(set(test_df['movieId'].unique()) - set(train_df['movieId'].unique()))
print(sorted(movies_not_included)[:10])
print()

not_included_df = test_df[test_df.movieId.isin(movies_not_included)].sort_values(by='movieId')
print(not_included_df.head())
print()

print("train set에 없고, test set에만 있는 영화 데이터 수: ", not_included_df.shape)

[49, 117, 137, 178, 241, 320, 359, 478, 488, 495]

       userId  movieId  rating  timestamp
29386     202       49     3.0  974925453
97066     604      117     3.0  832080636
99501     609      137     3.0  847221054
27959     191      178     1.0  829760898
98493     607      241     4.0  964744490

train set에 없고, test set에만 있는 영화 데이터 수:  (852, 4)


---

## 간단한 추천알고리즘 만들기

1. 랜덤으로 평점 예측하기
2. 영화 평균 평점기반 예측하기
3. 사용자 평균 평점기반 예측하기
4. Rule기반 영화 랭킹 예측하기

- 여기의 방법들은 추천 시스템이긴 하나 기초적인 통계를 활용하는 작업

### 1. 랜덤으로 평점 예측하기

In [15]:
# 0.5 - 5.0사이의 숫자를 예측해야할 평점 수 만큼 generate
ratings_range = np.arange(0.5, 5.5, step=0.5)
ratings_range

array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

In [16]:
import random
pred_random = [random.choice(ratings_range) for x in range(len(test_df))]
pred_random[:10]

[2.0, 3.0, 3.0, 2.5, 1.5, 0.5, 1.5, 3.0, 2.5, 2.0]

In [17]:
test_df['pred_ratings_random'] = pred_random

In [18]:
mse = mean_squared_error(y_true=test_df['rating'].values, y_pred=test_df['pred_ratings_random'].values)
rmse = np.sqrt(mse)

print('mse : ', mse)
print('rmse : ', rmse)

mse :  3.684413427211424
rmse :  1.9194825936203288


###  2. 영화 평균 평점기반 예측하기

    1. train set의 모든 영화에 대해서 평균 평점 구하기
    2. test set 예측할 때, train set의 영화 평균 평점 활용하기. 만약 없다면, random으로 선택하기.

In [19]:
train_movie_df = train_df.groupby('movieId').mean()

print(train_movie_df.shape)
print(train_movie_df.head())

(8938, 3)
             userId    rating     timestamp
movieId                                    
1        307.473373  3.893491  1.128439e+09
2        327.475610  3.396341  1.142893e+09
3        266.386364  3.454545  9.900434e+08
4        192.750000  2.250000  8.425133e+08
5        309.526316  3.039474  1.007415e+09


In [20]:
def avg_rating_prediction(training_set, x):
    if x in training_set.index:
        pred_rating = training_set.loc[x]['rating']
    else:
        pred_rating = random.choice(ratings_range)
    return pred_rating

In [21]:
test_df['pred_rating_movie'] = test_df['movieId'].apply(lambda x: avg_rating_prediction(train_movie_df, x))

test_df.head()

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_movie
99731,610,3527,5.0,1479545223,2.0,3.604167
97583,606,1250,3.5,1171376891,3.0,4.180556
38197,262,213,5.0,840310907,3.0,3.750000
11474,68,69406,3.0,1261622505,2.5,3.571429
34105,232,4728,3.0,1218166950,1.5,2.769231


In [22]:
mse = mean_squared_error(y_true=test_df['rating'].values, y_pred=test_df['pred_rating_movie'].values)
rmse = np.sqrt(mse)

print('mse : ', mse)
print('rmse : ', rmse)

mse :  1.058268263755602
rmse :  1.0287216648615902


### 3. 사용자 평균 평점기반 예측하기

1. train set의 모든 유저가 준 평균 평점
2. test set 예측할 때, 유저가 train set에서 준 평균 평점을 활용. 유저가 없을 경우 random 평점 적용

In [23]:
train_user_df = train_df.groupby('userId').mean()

print(train_user_df.shape)
print(train_user_df.head())

(610, 3)
             movieId    rating     timestamp
userId                                      
1        1891.168478  4.320652  9.649865e+08
2       70402.760000  3.940000  1.445715e+09
3        8394.733333  2.516667  1.306464e+09
4        1957.923077  3.631868  9.655941e+08
5         337.606061  3.636364  8.474351e+08


In [24]:
test_df['pred_rating_user'] = test_df['userId'].apply(lambda x: avg_rating_prediction(train_user_df, x))

test_df.head()

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_movie,pred_rating_user
99731,610,3527,5.0,1479545223,2.0,3.604167,3.678709
97583,606,1250,3.5,1171376891,3.0,4.180556,3.649718
38197,262,213,5.0,840310907,3.0,3.750000,2.925000
11474,68,69406,3.0,1261622505,2.5,3.571429,3.229331
34105,232,4728,3.0,1218166950,1.5,2.769231,3.242268


In [25]:
mse = mean_squared_error(y_true=test_df['rating'].values, y_pred=test_df['pred_rating_user'].values)
rmse = np.sqrt(mse)

print('mse : ', mse)
print('rmse : ', rmse)

mse :  0.8905889036428333
rmse :  0.9437101798978504


### 4. Rule 기반 영화 평점 예측하기

4-1. train set에 포함된 유저의 영화 평균 평점과 영화의 장르를 활용하여, 장르별 평균 평점 계산 -> test set의 영화 장르의 평균 평점으로 예측

In [26]:
# create user_movie matrix by only using train_df
train_user_movie_matrix = train_df.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)

In [27]:
train_user_movie_matrix

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,0.0,2.5,0.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,0.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
genres_df = movies_df['genres'].str.get_dummies(sep='|')
print(genres_df.shape)
genres_df = genres_df.loc[train_df.movieId.unique()]
print(genres_df.shape)
genres_df.head()

(9742, 20)
(8938, 20)


,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
5943,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2571,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
8958,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2322,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
2959,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0


In [29]:
# trainset에서 영화별 유저 평점 평균
train_movie_avg_ratings_df = train_user_movie_matrix.copy()
train_movie_avg_ratings_df = train_movie_avg_ratings_df.replace(0, np.NaN)
train_movie_avg_ratings_df = train_movie_avg_ratings_df.mean(axis = 1)

train_movie_avg_ratings_df.head()

movieId
1    3.893491
2    3.396341
3    3.454545
4    2.250000
5    3.039474
dtype: float64

In [30]:
# genres_df에서 해당 장르가 포함된 모든 영화 index를 가져와서, 해당 영화의 유저 평균 평점의 평균을 구해서 장르 평균 평점으로 활용
genres_avg_ratings_df = pd.DataFrame(index=genres_df.columns, columns=['avg_ratings'])

for genre in genres_avg_ratings_df.index:
    genre_avg_rating = train_movie_avg_ratings_df.loc[genres_df[genres_df[genre].isin([1])].index].mean()
    genres_avg_ratings_df.loc[genre]['avg_ratings'] = genre_avg_rating

genres_avg_ratings_df.head()

,avg_ratings
(no genres listed),3.33642
Action,3.11085
Adventure,3.230721
Animation,3.492258
Children,3.101232


In [31]:
def get_genre_avg_ratings(x):
    genres_list = movies_df.loc[x]['genres'].split('|')
    rating = 0
    for genre in genres_list:
        rating += genres_avg_ratings_df.loc[genre]['avg_ratings']
    
    return rating / len(genres_list)

In [32]:
tqdm.pandas()
test_df['pred_rating_genre'] = test_df['movieId'].progress_apply(lambda x: get_genre_avg_ratings(x))

100%|██████████████████████████████████████████████████████████████████████████| 20168/20168 [00:06<00:00, 2972.23it/s]


In [33]:
test_df.head()

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_movie,pred_rating_user,pred_rating_genre
99731,610,3527,5.0,1479545223,2.0,3.604167,3.678709,3.138325
97583,606,1250,3.5,1171376891,3.0,4.180556,3.649718,3.410377
38197,262,213,5.0,840310907,3.0,3.750000,2.925000,3.429093
11474,68,69406,3.0,1261622505,2.5,3.571429,3.229331,3.267870
34105,232,4728,3.0,1218166950,1.5,2.769231,3.242268,3.181480


In [34]:
mse = mean_squared_error(y_true=test_df['rating'].values, y_pred=test_df['pred_rating_genre'].values)
rmse = np.sqrt(mse)

print('mse : ', mse)
print('rmse : ', rmse)

mse :  1.1251906030478547
rmse :  1.0607500191128232


4-2. user의 평균 영화 평점을 normalize(min-max scale)해서 확인하기, 평점 측정 수, 표준편차 등 활용가능 

In [35]:
train_user_info_df = pd.DataFrame({
    'avg_ratings': train_df.groupby('userId')['rating'].mean(),
    'std_ratings': train_df.groupby('userId')['rating'].std(),
    'count_ratings': train_df.groupby('userId')['rating'].count()
})

train_user_info_df.head()

,avg_ratings,std_ratings,count_ratings
userId,,,
1,4.320652,0.836600,184
2,3.940000,0.820569,25
3,2.516667,2.127340,30
4,3.631868,1.317823,182
5,3.636364,1.084498,33


In [36]:
min_count = train_user_info_df['count_ratings'].min()
max_count = train_user_info_df['count_ratings'].max()
avg_count = train_user_info_df['count_ratings'].mean()

train_user_info_df['weights'] = train_user_info_df['count_ratings'].apply(lambda x: (x-avg_count)/(max_count-min_count))

In [37]:
train_user_info_df.head()

,avg_ratings,std_ratings,count_ratings,weights
userId,,,,
1,4.320652,0.836600,184,0.023995
2,3.940000,0.820569,25,-0.049718
3,2.516667,2.127340,30,-0.047400
4,3.631868,1.317823,182,0.023068
5,3.636364,1.084498,33,-0.046010


In [38]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(train_user_info_df)
df_normalized = pd.DataFrame(np_scaled, columns = train_user_info_df.columns, index=train_user_info_df.index)
df_normalized.head()

,avg_ratings,std_ratings,count_ratings,weights
userId,,,,
1,0.822227,0.393261,0.079740,0.079740
2,0.722617,0.385725,0.006027,0.006027
3,0.350156,1.000000,0.008345,0.008345
4,0.641984,0.619470,0.078813,0.078813
5,0.643161,0.509791,0.009736,0.009736


In [39]:
df_normalized['normalized_avg_ratings'] = df_normalized['avg_ratings'] * 5
df_normalized.head()

,avg_ratings,std_ratings,count_ratings,weights,normalized_avg_ratings
userId,,,,,
1,0.822227,0.393261,0.079740,0.079740,4.111134
2,0.722617,0.385725,0.006027,0.006027,3.613084
3,0.350156,1.000000,0.008345,0.008345,1.750779
4,0.641984,0.619470,0.078813,0.078813,3.209921
5,0.643161,0.509791,0.009736,0.009736,3.215803


In [40]:
test_df['pred_rating_normalized'] = test_df['userId'].apply(lambda x: df_normalized.loc[x]['normalized_avg_ratings'])
test_df.head()

,userId,movieId,rating,timestamp,pred_ratings_random,pred_rating_movie,pred_rating_user,pred_rating_genre,pred_rating_normalized
99731,610,3527,5.0,1479545223,2.0,3.604167,3.678709,3.138325,3.271208
97583,606,1250,3.5,1171376891,3.0,4.180556,3.649718,3.410377,3.233275
38197,262,213,5.0,840310907,3.0,3.750000,2.925000,3.429093,2.285047
11474,68,69406,3.0,1261622505,2.5,3.571429,3.229331,3.267870,2.683236
34105,232,4728,3.0,1218166950,1.5,2.769231,3.242268,3.181480,2.700164


In [41]:
mse = mean_squared_error(y_true=test_df['rating'].values, y_pred=test_df['pred_rating_normalized'].values)
rmse = np.sqrt(mse)

print('mse : ', mse)
print('rmse : ', rmse)

mse :  1.120579096060227
rmse :  1.05857408624065
